<a href="https://colab.research.google.com/github/Tanmay-Somani/Wh020_Linktree/blob/main/Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#book recommender system using collaborative clustering


In [120]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [121]:
books=pd.read_csv("sample_data/BX-Books.csv",sep=';',error_bad_lines=False ,encoding='latin-1') 

<ipython-input-121-b3d0ab71e29e>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books=pd.read_csv("sample_data/BX-Books.csv",sep=';',error_bad_lines=False ,encoding='latin-1')
b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw

In [122]:
books=books[['ISBN','Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher']]

In [123]:
books.rename(columns={
    "Book-Title":"title",
    "Books-Author":"author",
    "Year-Of-Publication":"year",
    "Image-URL-L":"img_url"
},inplace=True)

In [124]:
users=pd.read_csv("sample_data/BX-Users.csv",sep=';',error_bad_lines=False ,encoding='latin-1') 

<ipython-input-124-9ca8b7c8fe35>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  users=pd.read_csv("sample_data/BX-Users.csv",sep=';',error_bad_lines=False ,encoding='latin-1')


In [125]:
ratings=pd.read_csv("sample_data/BX-Book-Ratings.csv",sep=';',error_bad_lines=False ,encoding='latin-1') 

<ipython-input-125-252b9ded5838>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  ratings=pd.read_csv("sample_data/BX-Book-Ratings.csv",sep=';',error_bad_lines=False ,encoding='latin-1')


In collaborative system it is important for a user to give you ratings.

In [126]:
ratings.rename(columns={"User-ID":"user_id","Book-Rating":"rating"},inplace=True)

In [127]:
x=ratings["user_id"].value_counts()>200

In [128]:
y=x[x].index

In [129]:
ratings=ratings[ratings['user_id'].isin(y)]

In [130]:
ratings_with_books=ratings.merge(books,on="ISBN")

In [131]:
num_rating=ratings_with_books.groupby('title')['rating'].count().reset_index()

In [132]:
num_rating.rename(columns={"rating":"num_of_rating"},inplace=True)

In [133]:
final_rating=ratings_with_books.merge(num_rating,on ='title')

In [134]:
final_rating=final_rating[final_rating["num_of_rating"]>=50]

In [135]:
final_rating.sample(10)

,user_id,ISBN,rating,title,Book-Author,year,Publisher,num_of_rating
31305,30487,044023722X,8,A Painted House,John Grisham,2001,Dell Publishing Company,228
30608,43246,0385265700,0,The Book of Ruth (Oprah's Book Club (Paperback)),Jane Hamilton,1990,Anchor,116
57003,43246,0671021001,0,She's Come Undone (Oprah's Book Club),Wally Lamb,1998,Pocket,128
101105,101876,0316969443,0,Suzanne's Diary for Nicholas,James Patterson,2001,"Little, Brown",143
114131,130474,0440200563,0,Fine Things,Danielle Steel,1988,Dell,57
99082,7346,0312966970,8,Four To Score (A Stephanie Plum Novel),Janet Evanovich,1999,St. Martin's Paperbacks,98
25169,53174,0345313860,9,"The Vampire Lestat (Vampire Chronicles, Book II)",ANNE RICE,1986,Ballantine Books,123
126739,153718,0312983824,0,Faking It,Jennifer Crusie,2003,St. Martin's Paperbacks,97
71723,88693,0375705856,0,Plainsong (Vintage Contemporaries),KENT HARUF,2000,Vintage,62
25683,121726,0436140969,0,Foucault's Pendulum,Umberto Eco,1989,Secker &amp; Warburg,60


In [136]:
final_rating.drop_duplicates(['user_id','title'],inplace=True)

In [139]:
book_pivot=final_rating.pivot_table(columns='user_id',index='title',values='rating')

In [142]:
book_pivot.fillna(0,inplace=True)

In [143]:
from scipy.sparse import csr_matrix

In [144]:
book_sparse=csr_matrix(book_pivot)

In [146]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
model=